In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
def hermite(n,x):
    """
    HERMITE   Compute the Hermite polynomial of degree n.
    h = hermite(n, x) returns the Hermite polynomial of degree n in x.

    """

# Check whether n is a non-negative integer

    if np.logical_or(round(n)- n != 0, n < 0):
        #return print('n must be a positive integer')
        raise ValueError('n must be a positive integer') 
    hp = np.zeros(n+1)
    hp_l = np.zeros(n+1)
    #hp_l = hp.copy() # This to copies as opposed to sharing  memory
# Don't use hp_l = hp because it seems to make them equal for all time
    hp_u = np.zeros(n+1)
# if either of the first 2 polynomials are required just state them
    if n==0:
        hp = np.ones(1)
    elif n== 1:
        hp = [0.0,2.0]
    elif n>1:
        m = 2
        hp_l[0] = 1.0
        hp_u[1] = 2.0
        
        while m < n+1:
            hp = 2*(np.concatenate(([0],hp_u[0:n]))-(m-1)*hp_l) # Remember with indice addressing this mean from 0 to less than n
            m = m + 1
            # Now reset the polynomials
            hp_l = hp_u.copy()
            hp_u = hp.copy()
    hp = np.flipud(hp)
 # Now evaluate full vector h(n,x)
    res = np.polyval(hp,x)
    return res

 # Or alternative res calcuation that does not speed things up but is essential for Numba later
 #   y = np.zeros_like(x)
    
 #   for i in range(len(hp)):
 #       y = y * x + hp[i]
        
    # return y
    

In [3]:
hermite?

In [14]:
from numba import jit
@jit(nopython=True)
def hermite_fun(n,x):
#HERMITE   Compute the Hermite polynomial of degree n.
# h = hermite(n, x) returns the Hermite polynomial
#      of degree n in x.

# Check whether n is a non-negative integer

#     if np.logical_or(round(n)- n!= 0, n < 0):
#         #return print('n must be a positive integer')
#         raise ValueError('n must be a positive integer') 
    
    hp = np.zeros(n+1)
    hp_l = np.zeros(n+1)
    #hp_l = hp.copy() # This to copies as opposed to sharing  memory
# Don't use hp_l = hp because it seems to make them equal for all time
    hp_u = np.zeros(n+1)
# if either of the first 2 polynomials are required just state them
    if n==0:
        hp = np.ones(1)
    elif n== 1:
        hp = np.array([0.0,2.0])
    elif n>1:
        m = 2
        hp_l[0] = 1.0
        hp_u[1] = 2.0
        
        while m < n+1:
            hp = 2*(np.concatenate((np.array([0]), hp_u[0:n]))-(m-1)*hp_l) # Remember with indice addressing this mean from 0 to less than n
            m = m + 1
            # Now reset the polynomials
            hp_l = hp_u
            hp_u = hp
    hp = hp[::-1]
 # Now evaluate full vector h(n,x)
 # Can't use np.polyval in numba so explicitly expand poynomial calculator
    
    y = np.zeros_like(x)
    
    for i in range(len(hp)):
        y = y * x + hp[i]
        
    return y

In [5]:
import time # This is a package to do timing tests
ntest = 50
n_v = np.zeros(ntest+1)
n_v[ntest]=ntest
x_v_test = np.linspace(0,1,int(1e6))
t0 = time.time()
hermite(ntest,x_v_test)
t1 = time.time() 
np.polynomial.hermite.hermval(x_v_test,n_v)
t2 = time.time()
print('Built in time = ',t2-t1)
print('Developed code =', t1 -t0 )

Built in time =  0.7292876243591309
Developed code = 0.3051753044128418


In [ ]:
%prun hermite(ntest,x_v_test) # Nice call that allows you to see what takes the time

In [17]:
%%timeit -n 3 -r 10
hermite(ntest,x_v_test)

366 ms ± 13 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [15]:
%%timeit -n 3 -r 10
hermite_fun(ntest,x_v_test)

203 ms ± 56.5 ms per loop (mean ± std. dev. of 10 runs, 3 loops each)


In [9]:
%%timeit -n 10 -r 1
np.polynomial.hermite.hermval(x_v_test,n_v)

760 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 10 loops each)
